In [1]:
import csv
import boto3 
import pickle
import os
import json
import pandas as pd

In [2]:
########### Paths
# Path to where your want to save the resulting labels
rekog_results_dir = "C:/Users/User/Desktop/"

# Path to where your images are
rekog_images_dir = "C:/Users/User/Desktop/INSTAGRAM/"


In [3]:
# Read in your personal keys

personal_access_key = " "
secret_access_key = " "

# Initialize the boto client to access the Rekogniton api
client=boto3.client('rekognition','us-east-1', # or choose the best region for your work, 
                                               # e.g. the location of your S3 bucket if using that method to store images
                    aws_access_key_id = personal_access_key, 
                    aws_secret_access_key = secret_access_key) 

In [4]:
# Make a list of all the images in the rekog_data_dir you created
local_images = os.listdir(rekog_images_dir)
#local_images = [str(p) for p in rekog_images_dir]

In [5]:
##### Detect labels 

holder_labels = []

for imageFile in local_images:
    with open(rekog_images_dir + imageFile, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
    print('Detected labels for ' + imageFile)
    
    ## If no labels detected, still save the info:
    if len(response['Labels']) == 0:
        print ("No Labels Detected")
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        temp_dict["'Text'"] = False
        temp_dict["close_up"] = False
        temp_dict["Indoors"] = False
        temp_dict['Outdoors'] = False
        holder_labels.append(temp_dict)   
    
    else:
        label_counter = 1
        labels = []
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        has_text = False
        close_up = False
        indoors = False
        outdoors = False
        
        for label in response['Labels']:
            if label.get('Confidence') >= 70.0:
                labels.append(label.get('Name'))
            label_counter +=1 # update for the next label
        text_labels = ['Text', 'Flyer', 'Brochure', 'Paper', 'Advertisement', 'Paper'] # η λιστα με τα labels που δειχνουν text
    
        check = any(item in labels for item in text_labels)
        if check:
            has_text = True
        if 'Indoors' in labels:
            indoors = 'True'
        if 'Outdoors' in labels:
            outdoors = 'True'
        if 'Portrait' in labels:
            close_up = True
        
        temp_dict['Text'] = has_text
        temp_dict['close_up'] = close_up
        temp_dict['Indoors'] = indoors
        temp_dict['Outdoors'] = outdoors
        
        holder_labels.append(temp_dict)
        print(labels)

Detected labels for 121824333_1780170455456980_1945710384918790628_n.jpg
['Clothing', 'Apparel', 'Person', 'Human', 'Beanie', 'Cap', 'Hat']
Detected labels for 121968037_675216890074396_7745071755117364219_n.jpg
['Number', 'Text', 'Symbol', 'Home Decor', 'Field', 'Plot']
Detected labels for 121969359_1514124702115403_7612990989954934559_n.jpg
['Person', 'Human', 'Boy', 'Room', 'Indoors']
Detected labels for 121972372_384988435991392_6290850303008840320_n.jpg
['Clothing', 'Apparel', 'Person', 'Human', 'Mammal', 'Animal', 'Hat']
Detected labels for 122026324_625512811474271_6945540037480226640_n.jpg
['Text', 'Document', 'Page', 'Word']
Detected labels for 122044340_471148523818151_3074582108226996632_n.jpg
['Person', 'Human', 'Female', 'Face', 'Blonde', 'Girl', 'Woman', 'Teen', 'Kid', 'Child', 'Clothing', 'Apparel', 'Art', 'Smile', 'Dress']
Detected labels for 122044760_178946800402106_8604711366301652788_n.jpg
['Clothing', 'Apparel', 'Person', 'Human', 'Advertisement', 'Poster', 'Flag',

In [6]:
# Write out the results to a csv
with open(rekog_results_dir + 'labels_final_insta.csv', 'w', newline = '') as csvfile:
    fieldnames = ['image_id', 'Text', 'close_up','Indoors', 'Outdoors']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader() 
    for entry in holder_labels:
        writer.writerow(entry)

In [7]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)
face_labels = []

for imageFile in local_images:
    with open(rekog_images_dir + imageFile, 'rb') as image:
         response = client.detect_faces(Image={'Bytes': image.read()},Attributes=['ALL'])

    print('Detected labels for ' + imageFile)  
    if len(response['FaceDetails'])==0: # IF no faces are found
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        temp_dict["Child"] = False
        temp_dict['Smile'] = False
        temp_dict['Emotions'] =  []
        temp_dict['Female'] = False
        temp_dict['Male'] = False
        temp_dict['PeopleNumber'] = 0
        face_labels.append(temp_dict)
    else: 
        emotion_list = []
        has_child = False
        female = False
        male = False
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        temp_dict['PeopleNumber'] = len(response['FaceDetails'])
        
        for faceDetail in response['FaceDetails']:
            age = faceDetail['AgeRange']
            if age.get('High') <= 16:
                has_child = True
                
            smile = faceDetail['Smile']
            if smile.get('Value') == True:
                has_smile = True
            
            gender = faceDetail['Gender'].get('Value')
            if gender == 'Female':
                female = True
            if gender == 'Male':
                male = True
#             temp_dict['Emotions'] = faceDetail['Emotions']

            for emotion in faceDetail['Emotions']:
                if emotion.get('Confidence') >= 70.0:
                    emotion_list.append(emotion.get('Type'))
            emotion_list = list(set(emotion_list))
        
        temp_dict['Child'] = has_child
        temp_dict['Smile'] = has_smile
        temp_dict['Female'] = female
        temp_dict['Male'] =  male
        temp_dict['Emotions'] = emotion_list
        face_labels.append(temp_dict)

Detected labels for 121824333_1780170455456980_1945710384918790628_n.jpg
Detected labels for 121968037_675216890074396_7745071755117364219_n.jpg
Detected labels for 121969359_1514124702115403_7612990989954934559_n.jpg
Detected labels for 121972372_384988435991392_6290850303008840320_n.jpg
Detected labels for 122026324_625512811474271_6945540037480226640_n.jpg
Detected labels for 122044340_471148523818151_3074582108226996632_n.jpg
Detected labels for 122044760_178946800402106_8604711366301652788_n.jpg
Detected labels for 122058808_672982296979185_7654741695571408912_n.jpg
Detected labels for 122072259_123359139319109_6287889894631903949_n.jpg
Detected labels for 122080481_655182648462259_9018353764250614479_n.jpg
Detected labels for 122083011_3837163772984190_4494576759244229022_n.jpg
Detected labels for 122142920_2800224163596965_5430927296730905029_n.jpg
Detected labels for 122147852_186737109623965_5475099666055123242_n.jpg
Detected labels for 122154581_199183728287761_50436926890198

In [8]:
with open(rekog_results_dir + 'faces_final_insta.csv', 'w', newline = '') as csvfile:
    fieldnames = ['image_id', 'Child', 'Smile', 'Emotions', 'Female', 'Male', 'PeopleNumber'] # Εδώ μπορείς να κρατήσεις όποια attributes θέλεις. 
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in face_labels:
        writer.writerow(entry)